# web_data_flow walkthrough\n本 notebook 展示前端到后端再返回前端的 REST/JSON 链路。

In [ ]:
from fastapi.testclient import TestClient\nfrom web_data_flow.component import create_app, calc_order_summary, run_demo

In [ ]:
request_payload = {\n    "items": [\n        {"sku": "Keyboard", "qty": 2, "unit_price": 49.5},\n        {"sku": "Mouse", "qty": 1, "unit_price": 25.0},\n    ],\n    "tax_rate": 0.1,\n}\nrequest_payload

In [ ]:
summary = calc_order_summary(request_payload["items"], request_payload["tax_rate"])\nsummary

In [ ]:
app = create_app()\nclient = TestClient(app)\nresponse = client.post("/api/orders/summary", json=request_payload)\nresponse

In [ ]:
response.status_code, dict(response.headers), response.json()

In [ ]:
bad_payload = {"items": [{"sku": "Broken", "qty": 0, "unit_price": 10}], "tax_rate": 0.1}\nbad_response = client.post("/api/orders/summary", json=bad_payload)\nbad_response.status_code, bad_response.json()

In [ ]:
result = run_demo()\nresult["trace"], result["final_answer"]